In [ ]:
import os, sys
from loguru import logger

import numpy as np
import rasterio as rio

from matplotlib import pyplot as plt

from skimage.color import label2rgb, gray2rgb
from skimage.data import astronaut, camera
from skimage.graph import *
from skimage.segmentation import felzenszwalb, quickshift, slic, watershed


In [ ]:
sys.path.insert(1, '..')
import functions.fct_misc as fct

logger=fct.format_logger(logger)

In [ ]:
def print_im_segm_and_agg(image, segmented_image):
    out1=label2rgb(segmented_image, image, kind='avg', bg_label=0)

    print('Compute the graph')
    graph_segments=rag_mean_color(image, segmented_image)
    print('Merge segments')
    labels2=cut_normalized(segmented_image, graph_segments, 29)
    out2=label2rgb(labels2, image, kind='avg', bg_label=0)

    print('Plot results')
    fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(6, 8))

    ax[0].imshow(out1)
    ax[1].imshow(out2)

    for a in ax:
        a.axis('off')

    plt.tight_layout()

In [ ]:
WORKING_DIR='C:/Users/gwena/Documents/STDL/2_En_cours/occupation-toitures/02_Data/processed'

os.chdir(WORKING_DIR)

# image=camera()
# IMAGE_PATH='test_review/tile_list/tile_EGID_295135306.tif'
# IMAGE_PATH='test_review/tile_list/masked_images/tile_EGID_295135306_masked.tif'
IMAGE_PATH='roofs/normalized_masked_intensity_images/305963_normalized_intensity.tif'
with rio.open(IMAGE_PATH) as src:
    image=src.read(1)
    original_image=src.read()

original_image=np.transpose(original_image, [1, 2, 0])

In [ ]:
np.nan_to_num(image, copy=False, nan=999)

In [ ]:
scale=100
sigma=0.4
min_size=15
segmented_fel_image=felzenszwalb(image, scale=scale, sigma=sigma, min_size=min_size)

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(6, 8))
ax[0].imshow(image)
ax[1].imshow(segmented_fel_image)

In [ ]:
ratio=1
kernel_size=3
max_dist=6
segmented_qs_image=quickshift(image, ratio=ratio, kernel_size=kernel_size, max_dist=max_dist, convert2lab=False)

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(6, 8))
ax[0].imshow(image)
ax[1].imshow(segmented_qs_image)

In [ ]:
n_segments=20
compactness=0.01
max_num_iter=10
sigma=0
enforce_connectivity=False
min_size_factor=0.5
max_size_factor=1
slic_zero=False
start_label=1
segmented_slic_image=slic(image, n_segments=n_segments, compactness=compactness, max_num_iter=max_num_iter,
                sigma=sigma, enforce_connectivity=enforce_connectivity, min_size_factor=min_size_factor, max_size_factor=max_size_factor, slic_zero=slic_zero,
                start_label=start_label, mask=None, channel_axis=None)

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(6, 8))
ax[0].imshow(image)
ax[1].imshow(segmented_slic_image)

In [ ]:
# https://stackoverflow.com/questions/58812104/skimage-rag-merging-for-gray-images
image_three_bands=gray2rgb(image)
graph_slic=rag_mean_color(image_three_bands, segmented_slic_image, mode='similarity', sigma=50)
new_segmentation=cut_normalized(segmented_slic_image, graph_slic)

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(6, 8))

ax[0].set_title('RAG drawn with default settings')
lc = show_rag(segmented_slic_image, graph_slic, image_three_bands, ax=ax[0])
# specify the fraction of the plot area that will be used to draw the colorbar
fig.colorbar(lc, fraction=0.03, ax=ax[0])

ax[1].set_title('RAG drawn with grayscale image and viridis colormap')
lc = show_rag(segmented_slic_image, graph_slic, image_three_bands,
                    img_cmap='gray', edge_cmap='viridis', ax=ax[1])
fig.colorbar(lc, fraction=0.03, ax=ax[1])

for a in ax:
    a.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(6, 8))
ax[0].imshow(original_image)
ax[1].imshow(new_segmentation)

In [ ]:
new_segmentation = cut_threshold(segmented_slic_image, graph_slic, 10, in_place=False)
fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(6, 8))
ax[0].imshow(original_image)
ax[1].imshow(new_segmentation)

In [ ]:
markers=750
compactness=0.001
watershed_line=True
segmented_ws_image=watershed(image, markers=markers, compactness=compactness, watershed_line=watershed_line)

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(6, 8))
ax[0].imshow(image)
ax[1].imshow(segmented_ws_image)

In [ ]:
def _weight_mean_color(graph, src, dst, n):
    """Callback to handle merging nodes by recomputing mean color.

    The method expects that the mean color of `dst` is already computed.

    Parameters
    ----------
    graph : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `graph` to be merged.
    n : int
        A neighbor of `src` or `dst` or both.

    Returns
    -------
    data : dict
        A dictionary with the `"weight"` attribute set as the absolute
        difference of the mean color between node `dst` and `n`.
    """

    diff = graph.nodes[dst]['mean color'] - graph.nodes[n]['mean color']
    diff = np.linalg.norm(diff)
    return {'weight': diff}


def merge_mean_color(graph, src, dst):
    """Callback called before merging two nodes of a mean color distance graph.

    This method computes the mean color of `dst`.

    Parameters
    ----------
    graph : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `graph` to be merged.
    """
    graph.nodes[dst]['total color'] += graph.nodes[src]['total color']
    graph.nodes[dst]['pixel count'] += graph.nodes[src]['pixel count']
    graph.nodes[dst]['mean color'] = (graph.nodes[dst]['total color'] /
                                      graph.nodes[dst]['pixel count'])

image_three_bands=gray2rgb(image)
graph_slic=rag_mean_color(image_three_bands, segmented_slic_image, mode='similarity', sigma=25)
new_segmentation=cut_normalized(segmented_slic_image, graph_slic)

new_segmentation = merge_hierarchical(segmented_slic_image, graph_slic, thresh=35, rag_copy=False,
                                   in_place_merge=True,
                                   merge_func=merge_mean_color,
                                   weight_func=_weight_mean_color)

fig, ax = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(6, 8))
ax[0].imshow(original_image)
ax[1].imshow(new_segmentation)

In [ ]:
graph_slic._adj